In [1]:
! pip install faiss-cpu

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import ollama

In [3]:
texts = ["Antragsteller: Prof. Dr. Felix Wick",
         "Titel des Vorhabens: Agentische KI für die ingenieurwissenschaftliche Forschung",
         "Geplante Laufzeit: 12 Monate",
         "Arbeitsziel: Konzeption und Entwicklung des Prototyps eines digitalen Forschungsagenten, sowie dessen Erprobung in exemplarischen Anwendungsfällen der Werkstoffwissenschaft"]

In [4]:
embedder = SentenceTransformer("BAAI/bge-base-en-v1.5")

In [5]:
embs = embedder.encode(texts)

In [6]:
embs.shape

(4, 768)

In [7]:
embs = np.array(embs)
index = faiss.IndexFlatL2(embs.shape[1])
index.add(embs)

In [8]:
query = "Wer ist der Antragsteller?"

without context:

In [9]:
response = ollama.chat(model="mistral", messages=[
    {"role": "user", "content": f"Question:\n{query}"}
])
print(response["message"]["content"])

 Um diesen Satz zu übersetzen bedeutet "Wer ist der Antragsteller?" in etwa "Wer hat die Antragsstellung inne?" und kann also in folgender Weise zurückgegeben werden: Who is the applicant?


now with RAG:

In [10]:
query_emb = embedder.encode([query])
D, I = index.search(np.array(query_emb), k=2)
print("Nearest document:", texts[I[0][0]])

Nearest document: Antragsteller: Prof. Dr. Felix Wick


In [11]:
context = "\n\n".join(texts[i] for i in I[0])
context

'Antragsteller: Prof. Dr. Felix Wick\n\nGeplante Laufzeit: 12 Monate'

In [12]:
response = ollama.chat(model="mistral", messages=[
    {"role": "system", "content": "Use context to answer."},
    {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
])
print(response["message"]["content"])

 Der Antragsteller ist Prof. Dr. Felix Wick.
